In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to show the raw code."></form>''')

In [2]:
#increasing cell width with Jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import bqplot as bq

In [4]:
#read data and create dataframe
df = pd.read_csv("C:\\Users\Konrad\Desktop\DATA SCIENCE\Projekt Python - DataVirus\eda_final.csv", index_col=0)
df.describe().round(2)

age    gender    height    weight  systolic  diastolic  \
count  69542.00  69542.00  69542.00  69542.00  69542.00   69542.00   
mean      53.34      1.35    164.44     74.20    127.02      81.69   
std        6.76      0.48      7.86     14.25     17.07       9.84   
min       29.58      1.00    125.00     35.45     70.00      52.00   
25%       48.41      1.00    159.00     65.00    120.00      80.00   
50%       53.98      1.00    165.00     72.00    120.00      80.00   
75%       58.43      2.00    170.00     82.00    140.00      90.00   
max       64.97      2.00    207.00    200.00    240.00     180.00   

       cholesterol      gluc     smoke      alco   active   cardio       bmi  
count     69542.00  69542.00  69542.00  69542.00  69542.0  69542.0  69542.00  
mean          1.37      1.23      0.09      0.05      0.8      0.5     27.47  
std           0.68      0.57      0.28      0.23      0.4      0.5      5.18  
min           1.00      1.00      0.00      0.00      0.0      0.0     15.04  
25%           1.00      1.00      0.00      0.00      1.0      0.0     23.88  
50%           1.00      1.00      0.00      0.00      1.0      0.0     26.37  
75%           2.00      1.00      0.00      0.00      1.0      1.0     30.12  
max           3.00      3.00      1.00      1.00      1.0      1.0     59.52

In [5]:
#create function to categorize blood pressure
def blood_pressure_cat(diastolic):
    if diastolic >= 120:
        return "Crisis"
    elif (90 <= diastolic < 120):
        return "HBP_stage2"
    elif (80 <= diastolic < 90):
        return "HBP_stage1"
    else:
        return "Elevated/Normal"

In [6]:
#run category function on dataframe
df['HTN_stage'] = df['diastolic'].apply(lambda x: blood_pressure_cat(x))

In [7]:
#create function to categorize age
def age_category(age):
    if age < 40:
        return "30-39"
    elif (50 > age >= 40):
        return "40-49"
    elif (60 > age >= 50):
        return "50-59"
    else:
        return "60-69"
#run category function on dataframe
df['age_range'] = df['age'].apply(lambda x: age_category(x))

In [8]:
#creating handlers for data attibutes
gender_drop= widgets.Dropdown(options=[("Yes",1),
                                       ("No",0)],description="Include gender:",
                                         style= {'description_width': 'initial'})
run_button = widgets.Button(description="Show statistics",
                           button_style='success')

In [9]:
#function to show first plot
def show_plot_gender(b=None):
    gen=gender_drop.value
    plt.figure(figsize= (6,6))
    if gen == 0:
        ax =sns.countplot(y= 'HTN_stage', hue='gender', data= df, 
                          order=["Elevated/Normal","HBP_stage1", "HBP_stage2", "Crisis"])
        plt.title('Diastolic Blood Pressure Stages By Gender')
        plt.legend(["female", "male"])
    else:
        ax =sns.countplot(y= 'HTN_stage', data= df, 
                          order=["Elevated/Normal","HBP_stage1", "HBP_stage2", "Crisis"])
        plt.title('Diastolic Blood Pressure Stages Overview')   
    plt.ylabel('HTN stage')
    total = len(df['age_range'])
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y))

#function to show second plot
def show_plot_age(b=None):
    plt.figure(figsize= (6,6))
    sns.countplot(x= 'age_range', hue= 'HTN_stage', data= df, order=["30-39","40-49","50-59","60-69"])
    plt.title('Diastolic Blood Pressure Categories For Age Ranges', fontweight= 'bold', fontsize= 12)
    plt.xlabel('Age ranges')
    plt.ylabel('Number of Patients')
    plt.legend(["Elevated/Normal","HBP_stage1", "HBP_stage2", "Crisis"])

#function to show third plot    
def show_plot_cardio(b=None):
    plt.figure(figsize= (6,6))
    sns.countplot(x= 'HTN_stage', hue= 'cardio', 
                  data= df, order=["Elevated/Normal","HBP_stage1", "HBP_stage2", "Crisis"],
                 palette="Set1")
    plt.title('Diastolic Blood Pressure Categories With Cardio Disease Coexistance', fontweight= 'bold', fontsize= 12)
    plt.xlabel('Diastolic Blood Pressure Categories')
    plt.ylabel('Number of Patients')
    plt.legend(["cardio disease", "no cardio disease"])

In [12]:
#tabs
@run_button.on_click
def plot_on_click(b):
    out.clear_output(wait=True)
    with out:
        show_plot_gender()
        show_plot_age()
        show_plot_cardio()
        plt.show()

out=widgets.Output()
widget_box=widgets.VBox([gender_drop,run_button])

tab0 = widgets.Text('Age')
tab1 = widgets.Text('BMI')
tab2 = widgets.Text('Blood Pressure')
tab3 = widgets.Text('Systolic BP')
tab4 = widgets.HBox([widget_box,out])


tab_contents = ['Age', 'BMI', 'Blood Pressure', 'Systolic Blood Pressure', 'Diastolic Blood Pressure']
children = [tab0, tab1, tab2, tab3, tab4]
tab = widgets.Tab()
tab.children = children
for i in range(len(tab_contents)):
    tab.set_title(i, tab_contents[i])
tab